In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder\
    .appName("Recommender System Example")\
    .getOrCreate()

In [3]:
ratings = spark.read.load('../../data/movie-lens/ratings.csv', format='csv', header=True, inferSchema=True)
tags = spark.read.load('../../data/movie-lens/tags.csv', format='csv', header=True, inferSchema=True)
movies = spark.read.load('../../data/movie-lens/movies.csv', format='csv', header=True, inferSchema=True)
links = spark.read.load('../../data/movie-lens/links.csv', format='csv', header=True, inferSchema=True)

In [4]:
ratings.show(5)

+------+-------+------+----------+
|userId|movieId|rating| timestamp|
+------+-------+------+----------+
|     1|     31|   2.5|1260759144|
|     1|   1029|   3.0|1260759179|
|     1|   1061|   3.0|1260759182|
|     1|   1129|   2.0|1260759185|
|     1|   1172|   4.0|1260759205|
+------+-------+------+----------+
only showing top 5 rows



In [5]:
ratings.printSchema()

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- timestamp: integer (nullable = true)



In [6]:
movies.show(5)

+-------+--------------------+--------------------+
|movieId|               title|              genres|
+-------+--------------------+--------------------+
|      1|    Toy Story (1995)|Adventure|Animati...|
|      2|      Jumanji (1995)|Adventure|Childre...|
|      3|Grumpier Old Men ...|      Comedy|Romance|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|
|      5|Father of the Bri...|              Comedy|
+-------+--------------------+--------------------+
only showing top 5 rows



In [7]:
tags.show(5)

+------+-------+--------------------+----------+
|userId|movieId|                 tag| timestamp|
+------+-------+--------------------+----------+
|    15|    339|sandra 'boring' b...|1138537770|
|    15|   1955|             dentist|1193435061|
|    15|   7478|            Cambodia|1170560997|
|    15|  32892|             Russian|1170626366|
|    15|  34162|         forgettable|1141391765|
+------+-------+--------------------+----------+
only showing top 5 rows



In [8]:
links.show(5)

+-------+------+------+
|movieId|imdbId|tmdbId|
+-------+------+------+
|      1|114709|   862|
|      2|113497|  8844|
|      3|113228| 15602|
|      4|114885| 31357|
|      5|113041| 11862|
+-------+------+------+
only showing top 5 rows



In [9]:
training, test = ratings.randomSplit([0.8, 0.2])

In [10]:
from pyspark.ml.recommendation import ALS

In [17]:
als = ALS(maxIter=5, regParam=0.01, userCol="userId", itemCol="movieId", ratingCol="rating", coldStartStrategy="drop")

In [18]:
model = als.fit(training)

In [14]:
from pyspark.ml.evaluation import RegressionEvaluator

In [19]:
predictions = model.transform(test)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",
                                predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))

Root-mean-square error = 1.1155251910641162


In [20]:
predictions.show(5)

+------+-------+------+---------+----------+
|userId|movieId|rating|timestamp|prediction|
+------+-------+------+---------+----------+
|   232|    463|   4.0|955089443|   2.98064|
|   380|    463|   3.0|968949106| 2.9528782|
|   534|    463|   4.0|973377486| 3.0878317|
|   311|    463|   3.0|898008246| 3.2493916|
|   602|    471|   3.0|842357922| 4.2106757|
+------+-------+------+---------+----------+
only showing top 5 rows



In [21]:
# Generate top 10 movie recommendations for each user
userRecs = model.recommendForAllUsers(10)
# Generate top 10 user recommendations for each movie
movieRecs = model.recommendForAllItems(10)

In [22]:
userRecs.show(5)

+------+--------------------+
|userId|     recommendations|
+------+--------------------+
|   471|[[106438, 6.25312...|
|   463|[[121231, 5.54149...|
|   496|[[3328, 6.9461675...|
|   148|[[106438, 6.53105...|
|   540|[[2351, 8.691313]...|
+------+--------------------+
only showing top 5 rows



In [23]:
movieRecs.show(5)

+-------+--------------------+
|movieId|     recommendations|
+-------+--------------------+
|   1580|[[484, 5.761985],...|
|   5300|[[135, 7.4519887]...|
|   6620|[[375, 12.777893]...|
|   7340|[[411, 5.3641253]...|
|  32460|[[310, 7.4632096]...|
+-------+--------------------+
only showing top 5 rows



In [24]:
# Generate top 10 movie recommendations for a specified set of users
users = ratings.select(als.getUserCol()).distinct().limit(3)
userSubsetRecs = model.recommendForUserSubset(users, 10)

In [25]:
userSubsetRecs.show()

+------+--------------------+
|userId|     recommendations|
+------+--------------------+
|   471|[[106438, 6.25312...|
|   463|[[121231, 5.54149...|
|   148|[[106438, 6.53105...|
+------+--------------------+

